# Complessità computationale
In questo notebook verranno svolti alcuni esperimenti sulla complessità computazionale.
Nello, specifico verra analizzata e confrontata la complessità computazionale di alcune strutture dati fornite da
Python (list, set e deque) nell'effettuare le stesse operazioni.

Successivamente verrà richiesto di scrivere un algoritmo per ordinare una lista, la cui efficienza verrà confrontata con il metodo sort() di Python e con l'algoritmo insertion sort.

In [ ]:
# import dei moduli necessari
import matplotlib
import matplotlib.pyplot as plt
import time
import random
from typing import Iterable, Callable
from datetime import timedelta
from collections import deque
import copy

In [ ]:
# funzione per disegnare i grafici della complessità (usata in seguito)
def plot_time_comparison(sizes: list[int], *args: tuple[list[int], str]) -> None:
    fig, ax1 = plt.subplots(1,1)
    ax1.set_xlabel("Numero elementi (N)")
    ax1.set_ylabel("Tempo (millisecondi)")
    for times, label in args:
        ts = [t/1000 for t in times]
        ax1.plot(sizes, ts, label=label)
    plt.legend(loc="upper left")

# Strutture dati

La complessità di un'operazione su una struttura dati viene solitamente espressa in termini del numero di elementi contenuti. Usando la notazione O grande, la complessità di un'operazione sarà **O(F(N))**, dove **N** è il numero di elementi contentuti e **F(N)** una funzione di **N**.

Completare la funzione ```gen_problem_sizes```, al fine di generare l'elenco di dimensioni, ovvero il numero di elementi, con cui testare le strutture dati. Data una dimensione iniziale (*start_size*), una finale (*end_size*) e il numero voluto di punti (*num_points*), la funzione deve restituire una lista di interi equidistanziati nell'intervallo \[*start_size*, *end_size*\], estremi inclusi. Effettuare approssimazioni se necessario.

In [ ]:
def gen_problem_sizes(start_size: int, end_size:int, num_points:int) -> list[int]:
    step = (end_size - start_size)/(num_points - 1)
    return [round(start_size + step*i) for i in range(num_points)]

In [ ]:
gen_problem_sizes(5, 15, 3) # output: 5, 10, 15

Completare la funzione  ```gen_elements``` di modo che restituisca un set contenente numeri interi generati casulamente (ovviamente non duplicati). Il numero di interi che deve contenere il set è passato come parametro. Questi numeri interi verranno utlizzati per popolare la le strutture dati al fine di valutare la complessità delle operazioni per diverso numero di valori contenuti.

In [ ]:
def gen_elements(num_elm: int) -> set[int]:
    elm_set = set()
    while len(elm_set) < num_elm:
        elm = random.randint(0, num_elm*1000)
        elm_set.add(elm)
    return elm_set

In [ ]:
gen_elements(10) # output: 10 interi casuali

# Iterazione
Dati una lista (```list```) e un set (```set```) popolati con gli stessi elementi, quali sono le complessità computazionali nel leggere tutti gli elementi contenuti in funzione del loro numero?

Iniziamo a definire un intervallo di dimensioni per cui testare l'iterazione:

In [ ]:
start_size = 100  # dimensione iniziale
end_size = 20000  # dimenzione finale
num_points = 50   # numero di dimensioni da testare
sizes = gen_problem_sizes(start_size, end_size, num_points)
sizes

Generiamo ora le strutture dati da testare, delle dimensioni scelte:

In [ ]:
elements_list = [gen_elements(n) for n in sizes]        # genero un set di elementi per ciascuna dimensione
lists = [list(elements) for elements in elements_list]  # creo liste con gli elementi dei set
sets = [set(elements) for elements in elements_list]    # copio i set di elementi

Creiamo ora una funzione che, data una lista di strutture dati, per ognuna di esse itera su tutti gli elementi contenuti e restituisce in tempo impiegato per ciascuna struttura. Dato che con gli elementi su cui si itera non deve essere svolto alcun compito, si è aggiunta l'instuzione ```cont+=1``` invece di ```pass```, di modo che python non decida di saltare iterazioni per ottimizzare il tempo di esecuzione (questa è una tipica ottimizzazione di molti linguaggi).

Il tempo di esecuzione può essere influenzato da molti fattori (ad es. il pc è impegnato a fare altro), per cui ciascun test viene ripetuto più volte (```num_trials```) mediando il tempo di esecuzione.

In [ ]:
def test_iteration(data_structures: list[Iterable[int]], num_trials: int) -> list[int]:
    times = []
    for structure in data_structures:
        cont = 0
        t_start = time.perf_counter_ns()
        for t in range(num_trials):
            for val in structure:
                cont += 1
        duration = (time.perf_counter_ns() - t_start)/num_trials
        times.append(duration)
    return times     

A questo punto si hanno tutti gli elementi per eseguire il test. Prima di iniziare, si definisce quante volte ripetere ciasun test (```num_trials```), per avere un valore meno soggetto alle condizioni dell'elaboratore.

Eseguire la cella sottostante diverse volte, e notare che i grafici variano, anche in modo significativo, da una prova all'altra. Successivamente testare per valori di ```num_trials``` più alti e notare come questo rende i risultati meno casuali, delineando meglio il trend della complessità.

Secondo voi qual è, in notazione O grande, la complessità dell'iterazione sulle due strutture dati (```list``` e ```set```)?

In [ ]:
# test_iteration
num_trials = 5
list_times = test_iteration(lists, num_trials)
set_times = test_iteration(sets, num_trials)

# plot times
plot_time_comparison(sizes, (list_times, "List iteration"), (set_times, "Set iteration"))   

Dopo aver effettuato alcuni esperimenti, si intuisce che la complessità è lineare **O(n)** in entrambi i casi.

# Ricerca per valore
Sulle stesse strutture (```list``` e ```set```), testiamo ora il tempo impiegato per sapere se un elemento è contentuto nella struttura dati (l'operatore ```in``` in Python).

Definiamo, come prima, i parametri del test.

In [ ]:
start_size = 100
end_size = 20000
num_points = 50
sizes = gen_problem_sizes(start_size, end_size, num_points)

Creiamo le strutture dati delle diverse dimensioni

In [ ]:
elements_list = [gen_elements(n) for n in sizes]
lists = [list(elements) for elements in elements_list]
sets = [set(elements) for elements in elements_list]

Definiamo, come prima, una funzione che testi le strutture dati e salvi i tempi di esecuzione:

In [ ]:
def test_find(data_structures: Iterable[int], num_trials: int) -> list[int]:
    times = []
    for structure in data_structures:
        to_find = random.sample(list(structure), num_trials)
        cont = 0
        t_start = time.perf_counter_ns()
        for elm in to_find:
            if elm in structure:
                cont += 1
        duration = (time.perf_counter_ns() - t_start)/num_trials
        times.append(duration)
    return times  

Lanciamo gli esperimenti. Testare, come prima, per diversi valori di ```num_trials```. E' anche possibile cambiare gli altri parametri, come il numero di punti e l'intervallo di dimensioni.

Qual è la complessità dell'operazione per le due strutture dati?

In [ ]:
num_trials = 10
list_times = test_find(lists, num_trials)
set_times = test_find(sets, num_trials)

plot_time_comparison(sizes, (list_times, "List search"), (set_times, "Set search"))   

Dopo aver effettuato alcuni esperimenti, si intuisce che la complessità è lineare **O(n)** per le liste, mentre per i set è a tempo costante **O(1)**. Questo significa che non importa quanto il set sia grande, il tempo per sapere se un contiene un valore è sempre lo stesso.

Questo è uno dei vantaggi che si ottentono con i ```set```, sacrificando proprietà come l'ordinamento e la presenza di duplicati.

# Inserimento

Cambiamo ora le strutture dati in analisi. Passiamo a due implementazioni diverse del concetto di lista presenti in Python: ```list``` e  ```deque```.

Testiamo la complessità di inserimento nel caso di:
- inserimento in coda (in ultima posizione)
- inserimento in testa (in prima posizione)
- inserimento in posizione casuale

Iniziamo da definire i parametri dei test:

In [ ]:
start_size = 100
end_size = 20000
num_steps = 50
sizes = gen_problem_sizes(start_size, end_size, num_steps)

Creiamo le strutture dati da testare

In [ ]:
elements_list = [gen_elements(n) for n in sizes]
lists = [list(elements) for elements in elements_list]
deques = [deque(elements) for elements in elements_list]

Creiamo le funzioni che eseguono i diversi test:

In [ ]:
def test_insert_back(data_structures: Iterable[int], num_trials: int) -> list[int]:
    times = []
    for structure in data_structures:
        to_extend = [copy.copy(structure) for i in range(num_trials)]
        t_start = time.perf_counter_ns()
        for s in to_extend:
            s.append(1)
        duration = (time.perf_counter_ns() - t_start)/num_trials
        times.append(duration)
    return times

def test_insert_front(data_structures: Iterable[int], num_trials: int) -> list[int]:
    times = []
    for structure in data_structures:
        to_extend = [copy.copy(structure) for i in range(num_trials)]
        t_start = time.perf_counter_ns()
        for s in to_extend:
            s.insert(0, 1)
        duration = (time.perf_counter_ns() - t_start)/num_trials
        times.append(duration)
    return times


def test_insert_random(data_structures: Iterable[int], num_trials: int) -> list[int]:
    times = []
    for structure in data_structures:
        to_extend = [copy.copy(structure) for i in range(num_trials)]
        insert_positions = [random.randint(0, len(structure) - 1) for i in range(num_trials)]
        t_start = time.perf_counter_ns()
        for i in range(num_trials):
            to_extend[i].insert(insert_positions[i], 1)
        duration = (time.perf_counter_ns() - t_start)/num_trials
        times.append(duration)
    return times

Testiamo l'inserimento in coda, qual è la complessità?

In [ ]:
num_trials = 100
list_times_back = test_insert_back(lists, num_trials)
deque_times_back = test_insert_back(deques, num_trials)
plot_time_comparison(sizes, (list_times_back, "List back insertion"), (deque_times_back, "Deque back insertion"))

Testiamo l'inserimento in testa, qual è la complessità?

In [ ]:
num_trials = 100
list_times_front = test_insert_front(lists, num_trials)
deque_times_front = test_insert_front(deques, num_trials)
plot_time_comparison(sizes, (list_times_front, "List front insertion"), (deque_times_front, "Deque front insertion"))

Testiamo l'inserimento in posizione casuale, qual è la complessità?

In [ ]:
num_trials = 100
list_times_random = test_insert_random(lists, num_trials)
deque_times_random = test_insert_random(deques, num_trials)
plot_time_comparison(sizes, (list_times_random, "List random insertion"), (deque_times_random, "Deque random insertion")) 

L'inserimento in posizione casuale è lineare in entrambi i casi **O(N)**, anche se le deque sono più veloci come si vede per la diversa inclinazione della retta.
Quello che cambia, invece, è l'inserimento in testa e in coda, dove le ```deque``` sono ottimizzate per avere un tempo di inserimento lineare **O(1)**.

Quello che avviene in ```list``` è che, quando si inserisce un elemento, sposta avanti tutti gli altri per far spazio a quelli nuovi. In media è necessario spostare **N/2** elementi quando l'inserimento è casuale, mentre per l'inserimento in testa devono essere spostati tutti. Ma allora perchè l'inserimento in coda, dove non è necessario spostare elementi, rimane lineare?

### Amortized complexity

Qui le cose si complicano, perchè ```list``` per rendere efficiente inserire consecutivamente in coda, genera preventivamente le nuove posizioni, e quando effettivamente si inserisce si limita a copiare il valore. Questo viene fatto perchè, nonostante la complessità sia sempre **O(1)**, creare la posizione è molto più lento che soltanto copiare il valore, in quanto potrebbe essere necessario fare una richiesta al sistema operativo, aggiungendo dell'overhead per ogni inserimento. Facendolo una volta sola per più posizioni, si aggiunge l'overhead una volta sola.

Ma quando vengono generate queste nuove posizioni? Solitamente quando si finiscono quelle a disposizione. E quante ne vengono create? Un numero proporzionale alle dimenzioni della lista, usando il principio *più ne hai usate più ne userai*. Il tempo di questa operazione, quando viene svolta, è pertanto **O(M)** (lineare), dove **M** è proporzionale a **N**, secondo il principio sopra esposto, rendendo la complessità **O(N)**. Questa viene fatta una volta per i successivi **M** inserimenti che avengono in tempo costate, **O(1)**. Pertanto la complessità media (amortized complexity) è **O(M)/M = O(1)**.  

Creando una lista tramite costrutture, come facciamo nel codice di test, si sa già quanti elementi servono e non ne vengono creati di aggiuntivi. Quando però facciamo il primo inserimento, scatta l'allocazione delle posizioni vuote. Per questo percepiamo una complessità lineare.

Proviamo ora ad adattare la funzione di test di modo che si inseriscano 2 elementi e si misuri soltanto il tempo di inserimento del secondo. Il primo inserimento si occuperà dell'allocazione, mentre il secondo avrà già le posizioni allocate.

In [ ]:
def test_insert_back_adapted(data_structures: Iterable[int], num_trials: int) -> list[int]:
    times = []
    for structure in data_structures:
        to_extend = [copy.copy(structure) for i in range(num_trials)]
        for s in to_extend:
            s.append(1)
        t_start = time.perf_counter_ns()
        for s in to_extend:
            s.append(2)
        duration = (time.perf_counter_ns() - t_start)/num_trials
        times.append(duration)
    return times

start_size = 100
end_size = 20000
num_steps = 50
sizes = gen_problem_sizes(start_size, end_size, num_steps)

elements_list = [gen_elements(n) for n in sizes]
lists = [list(elements) for elements in elements_list]
deques = [deque(elements) for elements in elements_list]

num_trials = 100
list_times_back = test_insert_back_adapted(lists, num_trials)
deque_times_back = test_insert_back_adapted(deques, num_trials)
plot_time_comparison(sizes, (list_times_back, "List back insertion"), (deque_times_back, "Deque back insertion"))


In questo caso la complessità risulta costante **O(1)**.

# Accesso per posizione
Testiamo ora ```list``` e ```deque``` per quanto riguarda l'accesso per posizione.

Come prima, definiamo i parametri del test, creiamo le struttture dati e scriviamo la funzione di test.

In [ ]:
start_size = 100
end_size = 50000
num_steps = 100
sizes = gen_problem_sizes(start_size, end_size, num_steps)

In [ ]:
elements_list = [gen_elements(n) for n in sizes]
lists = [list(elements) for elements in elements_list]
deques = [deque(elements) for elements in elements_list]

In [ ]:
def test_indexing(data_structures: Iterable[int], num_trials: int) -> list[int]:
    times = []
    for structure in data_structures:
        positions = [random.randint(0, len(structure) - 1) for i in range(num_trials)]
        cont = 0
        t_start = time.perf_counter_ns()
        for pos in positions:
            cont += structure[pos]
        duration = (time.perf_counter_ns() - t_start)/num_trials
        times.append(duration)
    return times

Eseguiamo i test, qual è la complessità per le due strutture dati?

In [ ]:
num_trials = 100
list_times = test_indexing(lists, num_trials)
deque_times = test_indexing(deques, num_trials)

plot_time_comparison(sizes, (list_times, "List indexing"), (deque_times, "Deque indexing"))

La complessità è **O(1)** (costante) per ```list``` mentre è **O(N)** (lineare) per ```deque```. Questo avviene perchè quanto si usa ```deque``` è necessario scorrere tutti gli elementi per arrivare alla posizione voluta. Con ```list```, invece l'accesso è diretto ed effettuato in tempo costante.

# Sorting

Analizziamo ora l'algoritmo di ordinamento usato da Python quando si invoca ```sort()``` su una collezione ordinabile.

Esso verrà confrontato con un'implementazione dell' ```insertion sort``` e da un algoritmo scritto da voi.

In [ ]:
def insertion_sort(lst: list[int]):
    i = 1
    while i < len(lst):
        current = lst[i]
        j = i - 1
        while current < lst[j] and j >= 0:
            lst[j+1] = lst[j]
            j-=1
        lst[j+1] = current
        i+=1
    return lst

Implementare un algorimo di sorting, inventato da voi o scelto tra quelli mostrati a lezione:

In [ ]:
def your_sorting_alg(lst: list[int]):
    i = 1
    while i < len(lst):
        current = lst[i]
        j = i - 1
        while current < lst[j] and j >= 0:
            lst[j+1] = lst[j]
            j-=1
        lst[j+1] = current
        i+=1
    return lst

Definiamo i parametri e creiamo la funzione di test

In [ ]:
start_size = 100
end_size = 1000
num_steps = 10
sizes = gen_problem_sizes(start_size, end_size, num_steps)

In [ ]:
def test_sorting(lists: list[int], num_trials: int, sort_alg: Callable[[list[int]], list[int]]) -> list[int]:
    times = []
    for l in lists:
        to_sort = [copy.copy(l) for i in range(num_trials)]
        cont = 0
        t_start = time.perf_counter_ns()
        for l in to_sort:
            sort_alg(l)
        duration = (time.perf_counter_ns() - t_start)/num_trials
        times.append(duration)
    return times

Testare i 3 algoritmi, quali sono le complessità di quelli forniti? E di quello implementato da voi?

In [ ]:
num_trials = 5
lists = [list(gen_elements(n)) for n in sizes]

sort_times_python = test_sorting([copy.copy(l) for l in lists], num_trials, list.sort)
sort_times_insertion = test_sorting([copy.copy(l) for l in lists], num_trials, insertion_sort)
sort_times_your_alg = test_sorting([copy.copy(l) for l in lists], num_trials, your_sorting_alg)


plot_time_comparison(
    sizes, 
    (sort_times_python, "Python sort"),
    (sort_times_insertion, "Insertion sort"),
    (sort_times_your_alg, "Your algorithm"))

Python utilizza ```timsort``` come algoritmo di ordinamento. E' stato implementato per la prima volta nel 2002, appositamente per il linguaggio Python, e da allora è stato adottato da molti altri linguaggi. La sua complessità **NON** è costante, come potrebbe sembrare, ma mediamente **O(N)** (lineare) e **O(N log(N))** nei casi peggiori. Il fatto che dal grafico sembri constante, è dovuto al fatto che sia estremamente ottimizzato, anche nell'implementazione, rispetto agli altri algoritmi.

Per quanto riguarda l'```insersion sort``` la complessità è quadratica, **O(N^2)**, rendendono molto poco efficiente. Qual è la complessità del vostro algoritmo? Siete riusciti a scendere sotto **O(N^2)**?

# Conclusioni


In conclusione, questo laboratorio serve a dimostrare che una conoscenza delle strutture dati e degli algoritmi è essenziale anche nel caso in cui ci si limiti ad usare funzioni già pronte, come nel caso di Python. La scelta di un algorimo o struttura dati può migliorare o peggiorare drasticamente le performance del codice che si sviluppa.